In [93]:
import numpy
import scipy.special
import matplotlib.pyplot
%matplotlib inline
import pandas as pd


In [94]:
class neuralNetwork:
# инициализировать нейронную сеть
    def __init__(self, inputnodes, hiddennodes, outputnodes,learningrate):
        # задать количество узлов во входном, скрытом и выходном слое
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        # Матрицы весовых коэффициентов связей wih (между входным
        # и скрытым слоями)и who (между скрытым и выходным слоями).
        # Весовые коэффициенты связей между узлом i и узлом j
        # следующего слоя обозначены как w__i_j :
        # wll w21
        # wl2 w22 и т.д.
        self.wih = numpy.random.normal(0.0, pow(self.hnodes, -0.5),(self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.onodes, -0.5),(self.onodes, self.hnodes))
        self.lr = learningrate
        # использование сигмоиды в качестве функции активации
        self.activation_function = lambda x: scipy.special.expit(x)
        pass

    def train(self, inputs_list, targets_list):
        # преобразование списка входных значений
        # в двухмерный массив
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)
        # рассчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        # рассчитать входящие сигналы для выходного слоя
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        # ошибки выходного слоя =
        # (целевое значение - фактическое значение)
        output_errors = targets - final_outputs
        # ошибки скрытого слоя - это ошибки output_errors,
        # распределенные пропорционально весовым коэффициентам связей
        # и рекомбинированные на скрытых узлах
        hidden_errors = numpy.dot(self.who.T, output_errors)
        # обновить веса для связей между скрытым и выходным слоями
        self.who += self.lr * numpy.dot ((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        # обновить весовые коэффициенты для связей между
        # входным и скрытым слоями
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs) ), numpy.transpose(inputs))
        pass
    # опрос нейронной сети
    def query(self, inputs_list):
        # преобразовать список входных значений
        # в двухмерный массив
        inputs = numpy.array(inputs_list, ndmin=2).T
        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = numpy.dot(self.wih, inputs)
        # рассчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)
        # рассчитать входящие сигналы для выходного слоя
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)
        return final_outputs

In [95]:
# количество входных, скрытых и выходных узлов
input_nodes = 480
hidden_nodes = 80
output_nodes = 5
# коэффициент обучения
learning_rate = 0.2
# создать экземпляр нейронной сети
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate)

In [146]:
# загрузить в список тренировочный набор данных
# CSV-файла набора MNIST
training_data_file = pd.read_csv('EA25.csv')
training_data_file.drop(["camera", "mag_err", "flux", "flux_err"], axis = 1, inplace = True)


# training_data_list = training_data_file.readlines()
# del training_data_list[0]
# training_data_file.close()
# тренировка нейронной сети
# переменная epochs указывает, сколько раз тренировочный
#  набор данных используется для тренировки сети
epochs = 2
for е in range(epochs):
    # перебрать все записи в тренировочном наборе данных
    all = []
    a = []
for i in range(240):
    all.append(training_data_file['hjd'][i])
    
for j in range(240):
    a.append(training_data_file['mag'][j])

all_values = all + a

#     for i in range(200):
#          # получить список значений из записи, используя символы
#          # запятой (',') в качестве разделителей
#          if i % 6 != 0 or i % 6 != 2:
#                 del all_values[i]

    
         # масштабировать и сместить входные значения

inputs = []
for i in all_values:
    if i > 100:
        inputs.append((i-2456590)/8000.0 * 0.99 + 0.01)
    else:
        inputs.append((i-13)*0.99 + 0.01)
#         inputs.append((int(i)-13)*0.99 + 0.01) 
                
         # создать целевые выходные значения (все равны 0,01, за
         # исключением желаемого маркерного значения, равного 0,99)
targets = numpy.zeros(output_nodes) + 0.01
         # all_values[0] - целевое маркерное значение
         # для данной записи
targets[2] = 0.99
n.train(inputs, targets)
pass

In [147]:
# загрузить в список тестовый набор данных
# CSV-файла набора MNIST
test_data_file = pd.read_csv('EA25.csv')
test_data_file.drop(["camera", "mag_err", "flux", "flux_err"], axis = 1, inplace = True)

# тестирование нейронной сети
# журнал оценок работы сети, первоначально пустой

# перебрать все записи в тестовом наборе данных
all = []
a = []
for i in range(240):
    all.append(training_data_file['hjd'][i])
    
for j in range(240):
    a.append(training_data_file['mag'][j])
all_values = all + a
    # правильный ответ - первое значение
correct_label = 2
    # масштабировать и сместить входные значения
inputs = []
for i in all_values:
    if i > 100:
        inputs.append((i-2456590)/8000.0 * 0.99 + 0.01)
    else:
        inputs.append((i-13)*0.99 + 0.01)
    # опрос сети
outputs = n.query(inputs)
    # индекс наибольшего значения является маркерным значением
label = numpy.argmax(outputs)
    # присоединить оценку ответа сети к концу списка
if label == correct_label:
    scorecard.append(1)
else:
    scorecard.append(0)
    
if (label == 0):
    print("EW")
elif label == 1:
    print("EB")
elif label == 2:
    print("EA")
elif label == 3:
    print("ROT")
else:
    print("RRD")


pass
print(scorecard)

EA
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1]


# рассчитать показатель эффективности в виде
# доли правильных ответов
scorecard_array = numpy.asarray(scorecard)
print ("эффективность = ", scorecard_array.sum(), scorecard array.size)